In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

from collections import namedtuple, defaultdict, deque


In [26]:
input_text = au.read_txt_file_lines('input2.txt')
n_rows = len(input_text)
n_cols = len(input_text[0])
# for ii in range(1, n_rows):
#     assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 3 rows and 48 cols


In [47]:
data = []
machine = namedtuple('machine', ['con', 'but', 'act', 'mat', 'jol', 'arr'])

for l in input_text:
    con, tmp = l.split(']')
    buttons, jolts = tmp.split('{')

    con = np.array([1 if x == '#' else 0 for x in con.lstrip('[')])
    buttons = [tuple([int(k) for k in x.lstrip('(').rstrip(')').split(',')]) for x in buttons.split()]
    actions = [np.zeros(len(con), dtype=int) for _ in buttons]
    for i_b, b in enumerate(buttons):
        for i_c, c in enumerate(b):
            actions[i_b][c] = 1
    jolts = tuple([int(x) for x in jolts.rstrip('}').split(',')])
    mat = np.array(actions)
    arr = np.array(jolts)
    data.append(machine(con=con, but=buttons, act=actions, mat=mat, jol=jolts, arr=arr))

Sounds like a BFS search over graph .. ? Or Dijkstras [but with equal path lenght .?]?

we have the start and end node. 

In [48]:
def find_shortest_path(start, end, actions, mode='part1'):
    dict_visited = {tuple(start): 0}
    queue = deque([start])

    while len(queue) > 0:
        node = queue.popleft() 
        n_steps = dict_visited[tuple(node)]
        for a in actions:
            if mode == 'part1':
                new_node = np.abs(node - a)
            elif mode == 'part2':
                new_node = node + a

            if tuple(new_node) in dict_visited:
                continue 
            elif np.all(new_node == end):
                return n_steps + 1
            else:
                dict_visited[tuple(new_node)] = n_steps + 1
                queue.append(new_node)


total = 0
for m in tqdm(data):
    total +=find_shortest_path(np.zeros_like(m.con), m.con, m.act, mode='part1')
total
    




100%|██████████| 3/3 [00:00<00:00, 7069.05it/s]


7

# Part 2

In [49]:
# Naive BFS, takes too long!
total = 0
for m in tqdm(data):
    total += find_shortest_path(np.zeros_like(m.con), m.jol, m.act, mode='part2')
    print(total)
total

100%|██████████| 3/3 [00:00<00:00, 96.69it/s]

10
22
33


33

What's a smarter way .. 

the solution will be `jol = c1 * button_1 + c2 * button_2 .. `
(we can easily the find the max for each `c`, but still combinatiorially huge probably)
Let's break this down:

`jol_1 = c1 * b_1_1 + c2 * b_2_1 + ..`
`jol_2 = c1 * b_1_2 + c2 * b_2_2 + ..`

so we will have `len(buttons)` variables with `len(jolt)` equations. But there are many zeroes so probably it's solvable .. 

In [51]:
m = data[0]

m.mat, m.arr

(array([[0, 0, 0, 1],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [1, 0, 1, 0],
        [1, 1, 0, 0]]),
 array([3, 5, 4, 7]))

In [59]:
coefs = np.zeros(m.mat.shape[0], dtype=int)

k = 0
m.arr[k] - coefs[m.mat[:, 0].astype('bool')].sum()

np.int64(3)

In [57]:
m.mat[:, 0]

array([0, 0, 0, 0, 1, 1])

In [ ]:
def lin_system_solver(mat, arr):
    assert type(mat) == np.ndarray, type(mat)
    assert type(arr) == np.ndarray 
    assert mat.ndim == 2 and arr.ndim == 1

    assert mat.shape[1] == len(arr)
    n_coef, n_arr = mat.shape 
    print(f'There are {n_coef} coefs/buttons and {n_arr} jolts to match.')
    ## so coef[:] * mat[:, k] = arr[k]

    if n_arr == n_coef:
        print('solve with inv?')
    elif n_arr < n_coef:
        print('might be inf solutions')
    else:
        print('should be constrained')

    coefs = np.zeros(n_coef, dtype=int)


    for k, j in enumerate(arr):
        assert j - coefs[mat[:, k].astype('bool')].sum() == 0, (k, j, coefs)
    return coefs


lin_system_solver(m.mat, m.arr)

There are 6 coefs/buttons and 4 jolts to match.
might be inf solutions


AssertionError: (0, np.int64(3), array([0, 0, 0, 0, 0, 0]))

In [32]:
from numpy.linalg import lstsq

lstsq(np.array(m.act).T, np.array(m.jol))[0]

array([3.66666667, 1.33333333, 5.        , 2.33333333])

In [11]:
np.linalg.solve(a=np.array(m.act)[:, :4], b=np.array(m.jol)[:4])

LinAlgError: Singular matrix

In [ ]:
m.